# Mounting Storage using SAS Token Access-Key

## Azure Blob storage
- A service for storing large amounts of unstructured object data

## Requirements
- Can read data from public storage accounts without any additional settings
- To read data from a private storage account, must configure a Access Key or a Shared Access Signature (SAS)

## Mount Azure Blob storage containers to DBFS
- Can mount a Blob storage container or a folder inside a container to DBFS
- The mount is a pointer to a Blob storage container, so the data is never synced locally.
- DBFS uses the credential when creating the mount point to access mounted Blob storage container

In [0]:
# To mount a Blob storage container or a folder inside a container, use the following command
container_name="dbcontainer1"
storage_account_name="dbsaatin"
mount_name="dbcontainer1"
storage_account_access_key = "<specify>"
storage_account_access_key = "arwpsA/lhavTHyDP8hnIIzTrAahibFRzfiJ3FmFN2kH3AlDSsiF/f+7dN1w3CYXUj9AYtEUXorwCeWVGmUlJsw=="



In [0]:
# Can be either fs.azure.account.key.<storage-account-name>.blob.core.windows.net or fs.azure.sas.<container-name>.<storage-account-name>.blob.core.windows.net
conf_key = f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net"

if not any(mount.mountPoint == f"/mnt/{mount_name}" for mount in dbutils.fs.mounts()):
  try:
    dbutils.fs.mount(
    source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
    mount_point = f"/mnt/{mount_name}",
    extra_configs = {conf_key: storage_account_access_key}
  )
  except Exception as e:
    print("already mounted. Try to unmount first")

In [0]:
# Access files in your container as if they were local files, for example:
# python
file_name = "<specify>"
file_name = "genres.csv"

df1 = spark.read.csv(f"/mnt/{mount_name}/{file_name}")

display(df1)

_c0,_c1
1,A Cappella
2,Acid
3,Acoustic
4,Alternative
5,Ambient
6,Avantgarde
7,Bass
8,Beat
9,Bebob
10,Big Band


In [0]:
# Write back to Azure Blob Storage container
df1.write.json(f"/mnt/{mount_name}/{file_name}.json")

In [0]:
# Unmount a mount point
dbutils.fs.unmount(f"/mnt/{mount_name}")

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-208792214899783> in <module> 
 1 # Unmount a mount point 
 ----> 2 dbutils . fs . unmount ( f"/mnt/{mount_name}" ) 

 /databricks/python_shell/scripts/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 323 exc . __context__ = None 
 324 exc . __cause__ = None 
 --> 325 raise exc
 326 return f_with_exception_handling
 327 

 ExecutionError : An error occurred while calling o288.unmount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory not mounted: /mnt/dbcontainer1; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory not mounted: /mnt/dbcontainer1
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:128)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.unmount(DBUtilsCore.scala:837)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.unmount(DBUtilsCore.scala:848)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory not mounted: /mnt/dbcontainer1
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$deleteMount$1(MetadataManager.scala:461)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$modifyAndVerify$1(MetadataManager.scala:707)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:511)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.modifyAndVerify(MetadataManager.scala:696)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.deleteMount(MetadataManager.scala:465)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:103)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:97)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:96)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:96)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:297)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:256)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:53)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:80)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:80)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:49)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:434)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:240)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionConte

## Access Azure Blob storage using SAS

In [0]:
# Set up a SAS for a container:
sas_key_query_string = "<specify>"
sas_key_query_string = "sp=r&st=2021-06-28T12:32:28Z&se=2021-06-28T20:32:28Z&spr=https&sv=2020-02-10&sr=c&sig=1tHPQsrTiyYzJ0Wo7NXME328AqOk0Npnk2xsT5kC1QM%3D"


In [0]:
# Can be either fs.azure.account.key.<storage-account-name>.blob.core.windows.net or fs.azure.sas.<container-name>.<storage-account-name>.blob.core.windows.net
conf_key = f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net"


if not any(mount.mountPoint == f"/mnt/{mount_name}" for mount in dbutils.fs.mounts()):
    dbutils.fs.mount(
    source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
    mount_point = f"/mnt/{mount_name}",
    extra_configs = {conf_key: sas_key_query_string}
  )

In [0]:
dbutils.fs.ls(f"/mnt/")

Out[58]: [FileInfo(path='dbfs:/mnt/dbcontainer1/', name='dbcontainer1/', size=0)]

In [0]:
# Access files in your container as if they were local files, for example:
# python
file_name = "<specify>"
file_name = "genres.csv"

df1 = spark.read.csv(f"/mnt/{mount_name}/{file_name}")

display(df1)

_c0,_c1
1,A Cappella
2,Acid
3,Acoustic
4,Alternative
5,Ambient
6,Avantgarde
7,Bass
8,Beat
9,Bebob
10,Big Band
